In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature
# mlflow server --host 127.0.0.1 --port 8080

In [2]:
# Set our tracking server uri for logging
#export MLFLOW_TRACKING_URI=http://localhost:5000
#mlflow.set_tracking_uri(uri="http://localhost:8080")

In [3]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42
)

In [4]:
signature = infer_signature(train_x, train_y)

In [5]:
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define model architecture
    mean = np.mean(train_x, axis=0)
    var = np.var(train_x, axis=0)
    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["lr"], momentum=params["momentum"]
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            validation_data=(valid_x, valid_y),
            epochs=epochs,
            batch_size=64,
        )
        # Evaluate the model
        eval_result = model.evaluate(valid_x, valid_y, batch_size=64)
        eval_rmse = eval_result[1]

        # Log parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        # Log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [6]:
def objective(params):
    # MLflow will track the parameters and results for each run
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [7]:
space = {
    "lr": hp.loguniform("lr", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [8]:
# Create a new MLflow Experiment
mlflow.set_experiment("MLFLOW - Compare runs, choose a model, and deploy it to a REST API")

with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=8,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

2024/06/08 17:29:52 INFO mlflow.tracking.fluent: Experiment with name 'MLFLOW - Compare runs, choose a model, and deploy it to a REST API' does not exist. Creating a new experiment.
2024/06/08 17:29:53 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|rai

Epoch 1/3                                                            

46/46 [==============================] - 2s 14ms/step - loss: 11.2378 - root_mean_squared_error: 3.3523 - val_loss: 2.3508 - val_root_mean_squared_error: 1.5332

Epoch 2/3                                                            

46/46 [==============================] - 0s 5ms/step - loss: 1.9782 - root_mean_squared_error: 1.4065 - val_loss: 1.7533 - val_root_mean_squared_error: 1.3241

Epoch 3/3                                                            

46/46 [==============================] - 0s 6ms/step - loss: 1.5594 - root_mean_squared_error: 1.2487 - val_loss: 1.4923 - val_root_mean_squared_error: 1.2216

12/12 [==============================] - 0s 4ms/step - loss: 1.4923 - root_mean_squared_error: 1.2216

  0%|                          | 0/8 [00:03<?, ?trial/s, best loss=?]INFO:tensorflow:Assets written to: C:\Users\Pawan\AppData\Local\Temp\tmp9q398dyc\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\Pawan\AppData\Local\Temp\tmp9q398dyc\model\data\model\assets



Epoch 1/3                                                            

46/46 [==============================] - 1s 11ms/step - loss: 34.2417 - root_mean_squared_error: 5.8516 - val_loss: 32.8088 - val_root_mean_squared_error: 5.7279

Epoch 2/3                                                            

46/46 [==============================] - 0s 5ms/step - loss: 31.1829 - root_mean_squared_error: 5.5842 - val_loss: 29.8681 - val_root_mean_squared_error: 5.4652

Epoch 3/3                                                            

46/46 [==============================] - 0s 5ms/step - loss: 28.4077 - root_mean_squared_error: 5.3299 - val_loss: 27.1944 - val_root_mean_squared_error: 5.2148

12/12 [==============================] - 0s 4ms/step - loss: 27.1944 - root_mean_squared_error: 5.2148

 12%|▏| 1/8 [00:25<02:41, 23.12s/trial, best loss: 1.221590280532837]INFO:tensorflow:Assets written to: C:\Users\Pawan\AppData\Local\Temp\tmppxylgmec\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\Pawan\AppData\Local\Temp\tmppxylgmec\model\data\model\assets



...

Epoch 1/3                                                            

46/46 [==============================] - 1s 12ms/step - loss: 8.1669 - root_mean_squared_error: 2.8578 - val_loss: 1.6998 - val_root_mean_squared_error: 1.3037

Epoch 2/3                                                            

46/46 [==============================] - 0s 5ms/step - loss: 1.3264 - root_mean_squared_error: 1.1517 - val_loss: 0.9044 - val_root_mean_squared_error: 0.9510

Epoch 3/3                                                            

46/46 [==============================] - 0s 5ms/step - loss: 0.7378 - root_mean_squared_error: 0.8590 - val_loss: 0.6690 - val_root_mean_squared_error: 0.8179

12/12 [==============================] - 0s 4ms/step - loss: 0.6690 - root_mean_squared_error: 0.8179

 88%|▉| 7/8 [02:16<00:18, 18.97s/trial, best loss: 0.7424178123474121INFO:tensorflow:Assets written to: C:\Users\Pawan\AppData\Local\Temp\tmp306otuab\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\Pawan\AppData\Local\Temp\tmp306otuab\model\data\model\assets



100%|█| 8/8 [02:33<00:00, 19.19s/trial, best loss: 0.7424178123474121
INFO:tensorflow:Assets written to: C:\Users\Pawan\AppData\Local\Temp\tmpcqiz6cwi\model\data\model\assets


INFO:tensorflow:Assets written to: C:\Users\Pawan\AppData\Local\Temp\tmpcqiz6cwi\model\data\model\assets



Best parameters: {'lr': 0.06867219556438399, 'momentum': 0.4113604882502153}
Best eval rmse: 0.7424178123474121


In [9]:
import mlflow
import pandas as pd

# Load model as a PyFuncModel.
logged_model = 'runs:/4a59d74901e04d8c909d2233628a4b0e/model'
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Define the column names
columns = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides",
           "free sulfur dioxide", "total sulfur dioxide", "density", "pH", "sulphates", "alcohol"]

# Create the data as a list of lists (each inner list is a row)
data = [[7, 0.27, 0.36, 20.7, 0.045, 45, 170, 1.001, 3, 0.45, 8.8]]

# Create the DataFrame
df = pd.DataFrame(data, columns=columns)

# Predict using the loaded model
predictions = loaded_model.predict(df)

print(predictions)

c:\python3.8\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1/1 [==============================] - 0s 301ms/step
[[5.9544163]]
